<p style="
    padding-bottom: 10px;
    margin-bottom: 0;
    font-size: 44.96px;
    font-weight: 300;
    text-align: center;
    font-family: 'Didot', serif;
    color: #000000;  
    letter-spacing: 2px;  
">
    <b>Volatility-Based Supply &amp; Demand Levels Forecasting</b>
</p>
<hr style="
    height: 1px;
    border-width: 0;
    background-color: #ccc;
    margin-top: 2px;
    margin-bottom: 20px;
    box-shadow: 0px 1px 2px rgba(0, 0, 0, 0.1);
">


<h3 style="
    border-bottom: 1px solid #ccc;
    padding-bottom: 5px;
    margin-bottom: 10px;
    font-size: 18px;
    font-weight: 300;  
    font-family: 'Didot', serif;
    color: #333;
    letter-spacing: 1px;
">
    Table of Contents
</h3>

- [Introduction](#intro)<br><br>
- [Data Preprocessing](#preprocess)<br><br>
- [Annualized Volatility](#volatility)<br><br>
- [Supply &amp; Demand Levels ](#bands)<br><br>
- [Other Examples](#examples)<br><br>
- [Conclusion](#conclusion)<br><br>

<p id = 'intro'
   style="
    padding-bottom: 10px;
    margin-bottom: 0;
    font-size: 38px;
    font-weight: 300;
    text-align: left;
    font-family: 'Didot', serif;
    color: #000000;  
    letter-spacing: 2px;  
">
    <b>Introduction</b>
</p>

<hr style="
    height: 1px;
    border-width: 0;
    background-color: #ccc;
    margin-top: 2px;
    margin-bottom: 20px;
    box-shadow: 0px 1px 2px rgba(0, 0, 0, 0.1);
">

   - <p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    <b>Note:</b> The concepts explored in this notebook are inspired by the research paper <b><i><a href = "https://www.outspokenmarket.com/uploads/8/8/2/3/88233040/supply_and_demand_levels_forecasting_based_on_returns_volatility.pdf">Supply and Demand Levels Forecasting Based on Returns Volatility</a></i></b>, authored by <a href = "https://www.outspokenmarket.com/">Leandro Guerra</a>.
</p>


<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    The terms <i>"support"</i> and <i>"resistance"</i> represent two of the most basic foundations within those who look for opportunities in the markets based on the principles of Technical Analysis.
</p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    The whole idea behind the support and resistance levels is based on the law of supply and demand. When demand is greater than supply, we may expect prices to rise. While the opposite holds true when supply is greater than demand.
</p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    For many decades, technical analysts have developed different techniques for identifying these levels of support or resistance, looking for profit opportunities. One of the most common approaches is observing past price action, usually using candlestick charts, to identify levels where a trend reversal occurred. The figure below shows how these levels might be identified in regular technical analysis.
</p>

<center>
    <img src = "https://school.stockcharts.com/lib/exe/fetch.php?media=chart_analysis:candlesticks_and_resistance:candle6-resistdark-lu.png">
<p style = "font-size: 16px;
            font-family: 'Georgia', serif;
            text-align: center;
            margin-top: 10px;">Source: <a href = "https://school.stockcharts.com/doku.php?id=chart_analysis:candlesticks_and_resistance">StockCharts</a></p>
</center>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    The main issue, however, lies in the subjective nature of this kind of analysis. Different analysts may interpret support and resistance levels on the chart differently. Additionally, these supply and demand levels are inherently rooted in past price action, which doesn't guarantee their relevance for future market behavior.
</p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    In the research paper presented above, the author introduces the possibility of using the <b>returns</b>, due to its statistical properties, to compute <b>volatility</b>. Furthermore, it also proposes using volatility for <b>forecasting</b> supply and demand zones in a form that isn't subjective to personal interpretation of a price chart.    
</p>

In [1]:
!pip install yfinance -q # Installing Yahoo Finance for financial data retrieval

In [2]:
# Importing Libraries

# Data Handling
import pandas as pd
import numpy as np

# Financial Data Analysis
import yfinance as yf

# Data Visualization
import plotly.express as px
import plotly.graph_objs as go
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.io as pio
from IPython.display import display
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Statistics & Mathematics
import scipy.stats as stats
import statsmodels as sm
from scipy.stats import shapiro, skew
import math


# Hiding warnings
import warnings
warnings.filterwarnings("ignore")

<p id = 'preprocess'
   style="
    padding-bottom: 10px;
    margin-bottom: 0;
    font-size: 38px;
    font-weight: 300;
    text-align: left;
    font-family: 'Didot', serif;
    color: #000000;  
    letter-spacing: 2px;  
">
    <b>Data Preprocessing</b>
</p>

<hr style="
    height: 1px;
    border-width: 0;
    background-color: #ccc;
    margin-top: 2px;
    margin-bottom: 20px;
    box-shadow: 0px 1px 2px rgba(0, 0, 0, 0.1);
">

In [3]:
def load_and_preprocess(ticker):
    '''
    This function takes in a ticker symbol, which is used to retrieve historical data from Yahoo Finance.
    The attributes 'Returns', and the Adjusted Low, High, and Open are created.
    NaNs are filled with 0s
    '''

    df = yf.download(ticker)
    df['Returns'] = df['Adj Close'].pct_change(1)
    df['Adj Low'] = df['Low'] - (df['Close'] - df['Adj Close'])
    df['Adj High'] = df['High'] - (df['Close'] - df['Adj Close'])
    df['Adj Open'] = df['Open'] - (df['Close'] - df['Adj Close'])
    df = df.fillna(0)
    return df

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    For our first demonstration, we are going to use data for the <b>S&amp;P 500 (^GSPC)</b>.</p>

In [4]:
ticker = '^GSPC' # Defining a ticker symbol

df = load_and_preprocess(ticker) # Loading and Transforming Dataframe
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Returns,Adj Low,Adj High,Adj Open
Date,,,,,,,,,,
1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0,0.000000,17.660000,17.660000,17.660000
1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0,0.005663,17.760000,17.760000,17.760000
1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0,-0.002252,17.719999,17.719999,17.719999
1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0,-0.009594,17.549999,17.549999,17.549999
1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0,0.006268,17.660000,17.660000,17.660000
...,...,...,...,...,...,...,...,...,...,...
2023-09-20,4452.810059,4461.029785,4401.379883,4402.200195,4402.200195,3308450000,-0.009395,4401.379883,4461.029785,4452.810059
2023-09-21,4374.359863,4375.700195,4329.169922,4330.000000,4330.000000,3662340000,-0.016401,4329.169922,4375.700195,4374.359863
2023-09-22,4341.740234,4357.399902,4316.490234,4320.060059,4320.060059,3349570000,-0.002296,4316.490234,4357.399902,4341.740234


<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    Volatility, which is a measure of dispersion over a specific period, can be computed as the following:</p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
\begin{equation}
\text{vol} = \sigma\sqrt{T}
\end{equation}
</p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
Where:
    <br><br><br>
</p>

- <p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
Vol = Volatility for a certain interval of time.
</p>

- <p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
$\sigma$ = Standard deviation of returns.
</p>

- <p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
T = Time period.
</p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    The <b>daily</b> volatility can be computed with a moving window of $20$ days, as that is an approximation for the average number of trading days within a month.</p>
    
<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    By obtaining the daily volatility, we can then obtain the following metrics:</p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
\begin{equation}
\text{Annualized Volatility} = \text{Daily Volatility} \times \sqrt{252}
\end{equation}
</p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
\begin{equation}
\text{Monthly Volatility} = \frac{\text{Annualized Volatility}}{\sqrt{12}}
\end{equation}
</p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
\begin{equation}
\text{Weekly Volatility} = \frac{\text{Annualized Volatility}}{\sqrt{52}}
\end{equation}
</p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    In the following cell, we are going to create a new column in our dataframe called <code>Annualized_Vol</code> by obtaining the standard deviation of returns over a period of $T$ days and then multiply it by the $\sqrt{252}$.</p>

<p id = 'volatility'
   style="
    padding-bottom: 10px;
    margin-bottom: 0;
    font-size: 38px;
    font-weight: 300;
    text-align: left;
    font-family: 'Didot', serif;
    color: #000000;  
    letter-spacing: 2px;  
">
    <b>Annualized Volatility</b>
</p>

<hr style="
    height: 1px;
    border-width: 0;
    background-color: #ccc;
    margin-top: 2px;
    margin-bottom: 20px;
    box-shadow: 0px 1px 2px rgba(0, 0, 0, 0.1);
">

In [5]:
T = 20 # Time period for computing the standard deviation
df['Annualized_Vol'] = np.round(df['Returns'].rolling(T).std() * np.sqrt(252), 2)

In [16]:
# Creating a line plot to visualize historical volatility
lineplot = go.Scatter(
    x=df.index,
    y=df['Annualized_Vol'] * 100, # Annualized Volatility in %
    mode='lines',
    line=dict(color='darkblue', width=2.5),
    name = 'Annualized Volatility (%)')

layout = go.Layout(
    title={'text': f'<b>Annualized Volatility for {ticker} <br><i><sub>S&P 500</sub></i></b>',
           'x': 0.035, 'xanchor': 'left'},
            yaxis = dict(title = '<b>Annualized Volatility (%) </b>'),
            xaxis = dict(title = '<b>Date</b>'),
    template='seaborn',
    height = 450, width = 1000,
    showlegend=True,
    plot_bgcolor = '#F6F5F5',
    paper_bgcolor = '#F6F5F5',
    xaxis_rangeslider_visible=False)

fig = go.Figure(data=[lineplot], layout=layout)

fig.show()

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    It is possible to detect in the plot above periods of high volatility where the S&amp;P 500 experienced its highest points of fluctuation. The first period occurred in November 1929, during the turmoil of the <a href ="https://en.wikipedia.org/wiki/Wall_Street_Crash_of_1929">Wall Street Crash of 1929</a>. Other high-stress moments can be observed in November 1987 (Black Monday), November 2008 (Subprime Mortgage Crisis), and the most recent one in March 2020, during the onset of the COVID-19 pandemic.</p>
    
<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    As a baseline for interpreting volatility, we can follow the approach of making the <mark><b>assumption</b></mark> that the returns follow a normal distribution, although it may <b>not</b> always be the case. The paper's author suggests that further research could improve this aspect.</p>
    
<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
   With that being said, we interpret the annualized volatility plotted above as follows: Assuming a value of 15%, there is a <b>68.2% probability</b> that the security will experience a $\pm$ 15% price swing over the course of a year.</p>

<p id = 'bands'
   style="
    padding-bottom: 10px;
    margin-bottom: 0;
    font-size: 38px;
    font-weight: 300;
    text-align: left;
    font-family: 'Didot', serif;
    color: #000000;  
    letter-spacing: 2px;  
">
    <b>Supply &amp; Demand Levels</b>
</p>

<hr style="
    height: 1px;
    border-width: 0;
    background-color: #ccc;
    margin-top: 2px;
    margin-bottom: 20px;
    box-shadow: 0px 1px 2px rgba(0, 0, 0, 0.1);
">

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
   Given the concept of volatility, we are able to forecast the supply and demand levels for the next year according to the following:</p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
\begin{equation}
\text{Supply Band 1$\sigma$} = \text{(Annualized Volatility + 1)} \times \text{Reference Price}
\end{equation}
</p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
\begin{equation}
\text{Demand Band 1$\sigma$} = \text{Reference Price} \times \text{(1 - Annualized Volatility)}
\end{equation}
</p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
Where:
    <br><br><br>
</p>

- <p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
$\text{1$\sigma$}$ = $\text{1}$ Standard deviation from the mean.
</p>

- <p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
$\text{Reference Price}$ = Is the closing price in the last trading day of the year.
</p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
   We can also add other standard deviations. In case of two standard deviations from the mean, we can find the range in which there is a 95.45% probability that prices will be in, as given by the Gaussian Distribution below.</p>

<center>
    <img src = "https://supervisorbullying.com/wp-content/uploads/2022/09/normal-distribution-gaussian-distribution-1350x675px.webp">
<p style = "font-size: 16px;
            font-family: 'Georgia', serif;
            text-align: center;
            margin-top: 10px;">Gaussian/Normal Distribution. Source: <a href = "https://supervisorbullying.com/normal-distribution-gaussian-distribution/">Supervisor Bullying</a></p>
</center>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
   For computing the supply and demand levels for the 2 standard deviations from the mean, we must do the following:</p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
\begin{equation}
\text{Supply Band 2$\sigma$} = \text{(2 $\times$ Annualized Volatility + 1)} \times \text{Reference Price}
\end{equation}
</p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
\begin{equation}
\text{Demand Band 2$\sigma$} = \text{Reference Price} \times \text{(1 - Annualized Volatility $\times$ 2)}
\end{equation}
</p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    In the following cell, we are going to obtain the high and low bands for the <code>^GSPC</code>.</p>
    
<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    The variable <code>reference_year</code> will store the year used as reference to forecast the supply and demand levels for the subsequent year. These levels will be obtained by using as reference the Annualized Volatility and the Closing Price registered in the last trading day of the year of reference.</p>

In [7]:
# Yearly forecasting
reference_year = "2019" # Forecasting levels for 2020

High_Band_1std =  df.loc[reference_year]["Annualized_Vol"][-1]*df.loc[reference_year]["Adj Close"][-1] + df.loc[reference_year]["Adj Close"][-1]
Low_Band_1std =  df.loc[reference_year]["Adj Close"][-1] - df.loc[reference_year]["Annualized_Vol"][-1]*df.loc[reference_year]["Adj Close"][-1]

High_Band_2std =  2*df.loc[reference_year]["Annualized_Vol"][-1]*df.loc[reference_year]["Adj Close"][-1] + df.loc[reference_year]["Adj Close"][-1]
Low_Band_2std =  df.loc[reference_year]["Adj Close"][-1] - 2*df.loc[reference_year]["Annualized_Vol"][-1]*df.loc[reference_year]["Adj Close"][-1]

High_Band_3std =  3*df.loc[reference_year]["Annualized_Vol"][-1]*df.loc[reference_year]["Adj Close"][-1] + df.loc[reference_year]["Adj Close"][-1]
Low_Band_3std =  df.loc[reference_year]["Adj Close"][-1] - 3*df.loc[reference_year]["Annualized_Vol"][-1]*df.loc[reference_year]["Adj Close"][-1]


In [8]:
print(f'\nVolatility-Based Supply and Demand Levels for {ticker} in {int(reference_year) + 1}\n')
print(f'Supply Level 3σ: {High_Band_3std.round(2)}\n')
print(f'Supply Level 2σ: {High_Band_2std.round(2)}\n')
print(f'Supply Level 1σ: {High_Band_1std.round(2)}\n')
print('-' * 65, '\n')
print(f'Demand Level 1σ: {Low_Band_1std.round(2)}\n')
print(f'Demand Level 2σ: {Low_Band_2std.round(2)}\n')
print(f'Demand Level 3σ: {Low_Band_3std.round(2)}\n')


Volatility-Based Supply and Demand Levels for ^GSPC in 2020

Supply Level 3σ: 3909.24

Supply Level 2σ: 3683.09

Supply Level 1σ: 3456.93

----------------------------------------------------------------- 

Demand Level 1σ: 3004.63

Demand Level 2σ: 2778.47

Demand Level 3σ: 2552.32



<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    The output cell above gives us the specific supply and demand levels forecasted for the S&amp;500 (^GSPC) in 2020.</p>
    
<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    For better visualization, we can plot a Candlestick Chart displaying these levels, so we can see how prices reacted to them at that specific year.</p>

In [9]:
# Candlestick chart
candlestick = go.Candlestick(x = df.loc[str(int(reference_year) + 1)].index,
                             open = df.loc[str(int(reference_year) + 1)]['Adj Open'],
                             high = df.loc[str(int(reference_year) + 1)]['Adj High'],
                             low = df.loc[str(int(reference_year) + 1)]['Adj Low'],
                             close = df.loc[str(int(reference_year) + 1)]['Adj Close'],
                             increasing = dict(line=dict(color = 'red')),
                             decreasing = dict(line=dict(color = 'black')),
                             name = 'Candlesticks')


# Defining layout
layout = go.Layout(title = {'text': '<b>Volatility-Based Supply and Demand Levels<br><sup>&nbsp;&nbsp;&nbsp;&nbsp;<i>Annualized Forecast</i></sup></b>',
                            'x': .035, 'xanchor': 'left'},
                   yaxis = dict(title = '<b>Price (USD)</b>',
                               tickfont=dict(size=16)),
                   xaxis = dict(title = '<b>Date</b>'),
                   template = 'seaborn',
                   plot_bgcolor = '#F6F5F5',
                   paper_bgcolor = '#F6F5F5',
                   height = 850, width = 1000,
                   showlegend=False,
                   xaxis_rangeslider_visible = False)

# Defining figure
fig = go.Figure(data = [candlestick], layout = layout)

# Removing empty spaces (non-trading days)
dt_all = pd.date_range(start = df.index[0],
                       end = df.index[-1],
                       freq = "D")
dt_all_py = [d.to_pydatetime() for d in dt_all]
dt_obs_py = [d.to_pydatetime() for d in df.index]
dt_breaks = [d for d in dt_all_py if d not in dt_obs_py]

fig.update_xaxes(
    rangebreaks = [dict(values = dt_breaks)]
)

# Adding Supply and Demans Lines

# 1σ
fig.add_hline(y = High_Band_1std, line_width = 2, line_dash = "dot", line_color = "green")
fig.add_hline(y = Low_Band_1std, line_width = 2, line_dash = "dot", line_color = "red")

# 2σ
fig.add_hline(y = High_Band_2std, line_width = 4, line_dash = "dash", line_color = "green")
fig.add_hline(y = Low_Band_2std, line_width = 4, line_dash = "dash", line_color = "red")

# 3σ
fig.add_hline(y = High_Band_3std, line_width = 6, line_dash = "dashdot", line_color = "green")
fig.add_hline(y = Low_Band_3std, line_width = 6, line_dash = "dashdot", line_color = "red")

# Showing plot
fig.show()

<div style="border-left: 5px solid #888888; padding-left: 10px;">
    
<p style="
      padding-bottom: 10px;
      margin-bottom: 20px;
      font-size: 22px;
      font-weight: 400;  
      text-align: justify;
      font-family: 'Georgia', serif;  
      line-height: 1.6;
  ">
    • Dot lines represent one standard deviation (68.7% probability).
  </p>
  
<p style="
      padding-bottom: 10px;
      margin-bottom: 20px;
      font-size: 22px;
      font-weight: 400;  
      text-align: justify;
      font-family: 'Georgia', serif;  
      line-height: 1.6;
  ">
    • Dash lines represent two standard deviations (95.4% probability).
  </p>    

<p style="
      padding-bottom: 10px;
      margin-bottom: 20px;
      font-size: 22px;
      font-weight: 400;  
      text-align: justify;
      font-family: 'Georgia', serif;  
      line-height: 1.6;
  ">
    • Dash and dot lines represent three standard deviations (99.7% probability).
  </p>    
</div>


<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    Due to the COVID-19 pandemic, the year of 2020 was a very <b>unique</b> one. We can see that the S&amp;P 500 fell much under the 3$\sigma$ demand band in March, during the peak of the <b>panic selling</b>. </p>
    
<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    It's important to say, however, that prices didn't stay below that level for much longer. From April 6th and beyond, prices never traded below that level again for the rest of the year, rewarding those who took the risk of buying at that level.</p>
    
<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    It's also important to observe the 2$\sigma$ demand band serving as support and resistance throughout the months of April to May. More interestingly is to see the 1$\sigma$ demand band serving as pullback levels for prices during June, after a breakout in May.</p>
    
<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    By the end of the year, the S&amp;P 500 closed slightly higher than the 2$\sigma$ supply band.</p>

<p id = 'examples'
   style="
    padding-bottom: 10px;
    margin-bottom: 0;
    font-size: 38px;
    font-weight: 300;
    text-align: left;
    font-family: 'Didot', serif;
    color: #000000;  
    letter-spacing: 2px;  
">
    <b>Other Examples</b>
</p>

<hr style="
    height: 1px;
    border-width: 0;
    background-color: #ccc;
    margin-top: 2px;
    margin-bottom: 20px;
    box-shadow: 0px 1px 2px rgba(0, 0, 0, 0.1);
">

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    Considering the fact that 2020 was a distinguished year, it might be relevant to see the behavior between price and these supply and demand levels in different years and different securities. </p>
    
<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    Below, I will define a function called <code>plot_volatility_bands</code> that takes a <code>ticker</code> and a <code>reference_year</code> and returns a Plotly Candlestick Chart with the support and resistance levels, as well as prints containing the values for these supports and resistances.</p>

In [10]:
def plot_volatility_bands(ticker, reference_year):

    # Retrieving historical data and performing some preprocessing
    df = yf.download(ticker)
    df['Returns'] = df['Adj Close'].pct_change(1)
    df['Adj Low'] = df['Low'] - (df['Close'] - df['Adj Close'])
    df['Adj High'] = df['High'] - (df['Close'] - df['Adj Close'])
    df['Adj Open'] = df['Open'] - (df['Close'] - df['Adj Close'])
    df = df.fillna(0)

    # Obtaining the annualized volatility
    T = 20
    df['Annualized_Vol'] = np.round(df['Returns'].rolling(T).std()*np.sqrt(252), 2)

    # Calculating Bands
    High_Band_1std = df.loc[reference_year]["Annualized_Vol"][-1]*df.loc[reference_year]["Adj Close"][-1] + df.loc[reference_year]["Adj Close"][-1]
    Low_Band_1std = df.loc[reference_year]["Adj Close"][-1] - df.loc[reference_year]["Annualized_Vol"][-1]*df.loc[reference_year]["Adj Close"][-1]
    High_Band_2std =  2*df.loc[reference_year]["Annualized_Vol"][-1]*df.loc[reference_year]["Adj Close"][-1] + df.loc[reference_year]["Adj Close"][-1]
    Low_Band_2std =  df.loc[reference_year]["Adj Close"][-1] - 2*df.loc[reference_year]["Annualized_Vol"][-1]*df.loc[reference_year]["Adj Close"][-1]
    High_Band_3std =  3*df.loc[reference_year]["Annualized_Vol"][-1]*df.loc[reference_year]["Adj Close"][-1] + df.loc[reference_year]["Adj Close"][-1]
    Low_Band_3std =  df.loc[reference_year]["Adj Close"][-1] - 3*df.loc[reference_year]["Annualized_Vol"][-1]*df.loc[reference_year]["Adj Close"][-1]


    # Creating Candlestick chart
    candlestick = go.Candlestick(x = df.loc[str(int(reference_year) + 1)].index,
                             open = df.loc[str(int(reference_year) + 1)]['Adj Open'],
                             high = df.loc[str(int(reference_year) + 1)]['Adj High'],
                             low = df.loc[str(int(reference_year) + 1)]['Adj Low'],
                             close = df.loc[str(int(reference_year) + 1)]['Adj Close'],
                             increasing = dict(line=dict(color = 'red')),
                             decreasing = dict(line=dict(color = 'black')),
                             name = 'Candlesticks')


    # Defining layout
    layout = go.Layout(title = {'text': f'<b>Volatility-Based Supply and Demand Levels ({ticker})<br><br><sup>&nbsp;&nbsp;&nbsp;&nbsp;<i>Annualized Forecast - {str(int(reference_year) + 1)}</i></sup></b>',
                                'x': .035, 'xanchor': 'left'},
                       yaxis = dict(title = '<b>Price (USD)</b>',
                                   tickfont=dict(size=16)),
                       xaxis = dict(title = '<b>Date</b>'),
                       template = 'seaborn',
                       plot_bgcolor = '#F6F5F5',
                       paper_bgcolor = '#F6F5F5',
                       height = 850, width = 1000,
                       showlegend=False,
                       xaxis_rangeslider_visible = False)

    fig = go.Figure(data = [candlestick], layout = layout)

    # Fixing the empty spaces in the X-Axis
    dt_all = pd.date_range(start = df.index[0]
                           , end = df.index[-1]
                           , freq = "D")
    dt_all_py = [d.to_pydatetime() for d in dt_all]
    dt_obs_py = [d.to_pydatetime() for d in df.index]

    dt_breaks = [d for d in dt_all_py if d not in dt_obs_py]

    fig.update_xaxes(
        rangebreaks = [dict(values = dt_breaks)]
    )

    # 1σ
    fig.add_hline(y = High_Band_1std, line_width = 2, line_dash = "dot", line_color = "green")
    fig.add_hline(y = Low_Band_1std, line_width = 2, line_dash = "dot", line_color = "red")

    # 2σ
    fig.add_hline(y = High_Band_2std, line_width = 4, line_dash = "dash", line_color = "green")
    fig.add_hline(y = Low_Band_2std, line_width = 4, line_dash = "dash", line_color = "red")

    # 3σ
    fig.add_hline(y = High_Band_3std, line_width = 6, line_dash = "dashdot", line_color = "green")
    fig.add_hline(y = Low_Band_3std, line_width = 6, line_dash = "dashdot", line_color = "red")

    fig.show()

    # Printing Supply and Demand Levels
    print(f"\nVolatility-Based Supply and Demand Levels for {ticker} in {int(reference_year) + 1}\n")
    print(f"Supply Level 3σ: {High_Band_3std.round(2)}\n")
    print(f"Supply Level 2σ: {High_Band_2std.round(2)}\n")
    print(f"Supply Level 1σ: {High_Band_1std.round(2)}\n")
    print('-' * 65, '\n')
    print(f"Demand Level 1σ: {Low_Band_1std.round(2)}\n")
    print(f"Demand Level 2σ: {Low_Band_2std.round(2)}\n")
    print(f"Demand Level 3σ: {Low_Band_3std.round(2)}\n")

In [11]:
plot_volatility_bands('AAPL', '2021') # Forecasting supply and demand levels for 2022

[*********************100%%**********************]  1 of 1 completed



Volatility-Based Supply and Demand Levels for AAPL in 2022

Supply Level 3σ: 334.01

Supply Level 2σ: 281.27

Supply Level 1σ: 228.53

----------------------------------------------------------------- 

Demand Level 1σ: 123.06

Demand Level 2σ: 70.32

Demand Level 3σ: 17.58



<div style="border-left: 5px solid #888888; padding-left: 10px;">
    
<p style="
      padding-bottom: 10px;
      margin-bottom: 20px;
      font-size: 22px;
      font-weight: 400;  
      text-align: justify;
      font-family: 'Georgia', serif;  
      line-height: 1.6;
  ">
    • Apple (APPL) traded between the 1$\sigma$ supply and demand levels throughout the year during 2022, without much price action.
  </p>
</div>


In [12]:
plot_volatility_bands('TSLA', '2022') # Forecasting supply and demand levels for 2023

[*********************100%%**********************]  1 of 1 completed



Volatility-Based Supply and Demand Levels for TSLA in 2023

Supply Level 3σ: 396.64

Supply Level 2σ: 305.49

Supply Level 1σ: 214.33

----------------------------------------------------------------- 

Demand Level 1σ: 32.03

Demand Level 2σ: -59.13

Demand Level 3σ: -150.28



<div style="border-left: 5px solid #888888; padding-left: 10px;">
    
<p style="
      padding-bottom: 10px;
      margin-bottom: 20px;
      font-size: 22px;
      font-weight: 400;  
      text-align: justify;
      font-family: 'Georgia', serif;  
      line-height: 1.6;
  ">
    • In 2023, for Tesla (TSLA), the 1$\sigma$ supply band worked as a resistance level during February and March. There was a breakout of this level only in June, and this band worked as a support in August.
  </p>
</div>


In [13]:
plot_volatility_bands('BTC-USD', '2018') # Forecasting supply and demand levels for 2019

[*********************100%%**********************]  1 of 1 completed



Volatility-Based Supply and Demand Levels for BTC-USD in 2019

Supply Level 3σ: 12163.78

Supply Level 2σ: 9356.75

Supply Level 1σ: 6549.73

----------------------------------------------------------------- 

Demand Level 1σ: 935.68

Demand Level 2σ: -1871.35

Demand Level 3σ: -4678.38



<div style="border-left: 5px solid #888888; padding-left: 10px;">
    
<p style="
      padding-bottom: 10px;
      margin-bottom: 20px;
      font-size: 22px;
      font-weight: 400;  
      text-align: justify;
      font-family: 'Georgia', serif;  
      line-height: 1.6;
  ">
    • For Bitcoin (BTC) in 2018, the 3$\sigma$ supply level worked as a resistance level at least three times during that year. The 2$\sigma$ supply level worked as support until a breakout happened in September 24<sup>th</sup>. This same level served as a resistance zone during October and November, taking prices to another downtrend towards the 1$\sigma$ supply level.
  </p>
</div>


In [14]:
plot_volatility_bands('MSFT', '2022') # Forecasting supply and demand levels for 2023

[*********************100%%**********************]  1 of 1 completed



Volatility-Based Supply and Demand Levels for MSFT in 2023

Supply Level 3σ: 431.13

Supply Level 2σ: 366.82

Supply Level 1σ: 302.51

----------------------------------------------------------------- 

Demand Level 1σ: 173.88

Demand Level 2σ: 109.57

Demand Level 3σ: 45.26



<div style="border-left: 5px solid #888888; padding-left: 10px;">
    
<p style="
      padding-bottom: 10px;
      margin-bottom: 20px;
      font-size: 22px;
      font-weight: 400;  
      text-align: justify;
      font-family: 'Georgia', serif;  
      line-height: 1.6;
  ">
    • For Microsoft (MSFT) in 2023, the highest traded price was slightly under the 2$\sigma$ supply level.
  </p>
</div>


<p id = 'conclusion'
   style="
    padding-bottom: 10px;
    margin-bottom: 0;
    font-size: 38px;
    font-weight: 300;
    text-align: left;
    font-family: 'Didot', serif;
    color: #000000;  
    letter-spacing: 2px;  
">
    <b>Conclusion</b>
</p>

<hr style="
    height: 1px;
    border-width: 0;
    background-color: #ccc;
    margin-top: 2px;
    margin-bottom: 20px;
    box-shadow: 0px 1px 2px rgba(0, 0, 0, 0.1);
">

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    In this notebook, we explored a different approach to the support and resistance trading method that is so common in Technical Analysis.</p>
    
<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    The approach presented by author <a href = "https://www.outspokenmarket.com/">Leandro Guerra</a> in the <b><i><a href = "https://www.outspokenmarket.com/uploads/8/8/2/3/88233040/supply_and_demand_levels_forecasting_based_on_returns_volatility.pdf">Supply and Demand Levels Forecasting Based on Returns Volatility</a></i></b> paper brings a new outlook to this method, using the historical volatility for forecasting relevant support and resistance levels for the next year, month, and week.</p>
    
<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    There are several approaches to working with these levels. A trader might want to buy or sell an asset when it's close to the supply or resistance levels, as well as trade options looking for profits.</p>
    
<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    It might also be a good idea to use these supply and demand levels as input for <i>Machine Learning</i> models to help in the decision-making process for trading.</p>
    
<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    Of course, the methodology presented here isn't a <i>Holy Grail</i>. The assumption that returns are normally distributed, for instance, is not quite true for every single security, and this is definitely a valid area for improvement.</p>
    
<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    It's also important to say that this notebook and the ideas presented here don't serve the purpose of being a trading nor investment advice!</p>
    
<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">
    I strongly recommend reading the research paper that inspired this Notebook. It offers a deeper dive into the concepts covered here and explores additional topics like weekly supply and demand levels forecasting.</p>

<hr style="
    height: 1px;
    border-width: 0;
    background-color: #ccc;
    margin-top: 2px;
    margin-bottom: 20px;
    box-shadow: 0px 1px 2px rgba(0, 0, 0, 0.1);
">

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">If you liked this notebook, you may wish to read my other related works on Data Science &amp; Machine Learning applied to finance:</p>

- <p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
"><a href = "https://www.kaggle.com/code/lusfernandotorres/data-science-for-financial-markets"><b>🤑 Data Science for Financial Markets 📈💰</b></a></p>

- <p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
"><a href = "https://www.kaggle.com/code/lusfernandotorres/trading-with-machine-learning-classification"><b>🤖 Trading with Machine Learning: Classification</b></a></p>

- <p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
"><a href = "https://www.kaggle.com/code/lusfernandotorres/trading-with-machine-learning-regression"><b>🤖 Trading with Machine Learning: Regression</b></a></p>

- <p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
"><a href = "https://www.kaggle.com/code/lusfernandotorres/trading-with-machine-learning-weekly-returns"><b>🤖 Trading with Machine Learning: Weekly Returns</b></a></p>

- <p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
"><a href = "https://www.kaggle.com/code/lusfernandotorres/trading-with-machine-learning-follow-the-trend"><b>🤖 Trading with Machine Learning: Follow the Trend</b></a></p>
    
- <p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
"><a href = "https://www.kaggle.com/code/lusfernandotorres/trading-with-machine-learning-ensemble-methods"><b>🤖 Trading with Machine Learning: Ensemble Methods</b></a></p>

- <p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
"><a href = "https://www.kaggle.com/code/lusfernandotorres/s-p500-volatility-arch-vs-garch-models"><b>📉🔎S&amp;P500 Volatility: ARCH vs GARCH Models</b></a></p>

<p style="
    padding-bottom: 10px;
    margin-bottom: 20px;
    font-size: 22px;
    font-weight: 400;  
    text-align: justify;
    font-family: 'Georgia', serif;  
    line-height: 1.6;
">Thank you for reading!</p>

#### <hr style="border: 0; height: 1px; border-top: 0.85px solid #b2b2b2">
<div style="text-align: left; color: #8d8d8d; padding-left: 15px; font-size: 14.25px;">
    Luis Fernando Torres, 2023 <br><br>
    Let's connect!🔗<br>
    <a href="https://www.linkedin.com/in/luuisotorres/">LinkedIn</a> • <a href="https://medium.com/@luuisotorres">Medium</a> • <a href = "https://huggingface.co/luisotorres">Hugging Face</a><br><br>
</div>